# Source code to extract data for below Attributes:
Population    
Per_Capita_Expenditure (PCE)
GDP   
Consumer_Price_Index   
Currency_Conversion    
Website: http://iresearch.worldbank.org/PovcalNet/data.aspx

Importing all the required packages and modules

In [1]:
# pip install jupyter notebook==5.7.8 tornado==4.5.3
# pip install requests_htmlfrom bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from functools import reduce
from bs4 import BeautifulSoup
import nest_asyncio
import pandas as pd
nest_asyncio.apply()
from requests_html import HTMLSession
import pandas as pd
import time

Web Scraping Code for population attribute usiong HTMLSession

In [2]:
#function to extract data from html format
def extract_data_population(html_data):
    table = html_data.find('body').find('table')
    header = [str(country.text.strip()) for country in table.find('thead').find_all('td')]
    header.pop(-1)
    header[0]="Countries"
    table1 = html_data.find_all('tbody')[1]
    data = []
    data = {country: [] for country in header}
    #extracting row
    for row in table1.find_all("tr"):
        #extractinmg columns
        cols = [col.text for col in row.find_all('td')]
        if len(cols)==11:
            for i, col in enumerate(cols):
                data[header[i]].append(col)
    return pd.DataFrame(data) #converting python dictionary to data frame using PANDAS

#function to get the source page(html format) and parse it to beautifull soup
def scrap_data_population(url):
    # create an HTML Session object
    session = HTMLSession()
    # Use the object above to connect to needed webpage
    resp = session.get(url)
    # Run JavaScript code on webpage
    resp.html.render()
    soup = BeautifulSoup(resp.html.html, "lxml")
    return extract_data_population(soup)

#population function call
def Population():
    url="http://iresearch.worldbank.org/PovcalNet/data.aspx"
    population=scrap_data_population(url)
    df_final_population=pd.melt(population, id_vars='Countries', value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
    df_final_population.columns = ['Country','Year','Population']
    return df_final_population

Web Scraping for attributes using Selenium (code)

In [8]:
#function to extract data from html format
def extract_data(html_data):
    table = html_data.find("thead")
    header = [str(country.text.strip()) for country in table.find_all('td')]
    header.pop(-1)
    header[0]="Countries"
    table1 = html_data.find_all('tbody')[1]
    data = []
    data = {country: [] for country in header}
    
    #extracting row
    for row in table1.find_all("tr"):
        #extarcting column
        cols = [col.text for col in row.find_all('td')]
        if len(cols)>1:
            for i, col in enumerate(cols):
                data[header[i]].append(col)
    df=pd.DataFrame(data) #converting python dictionary to data frame using PANDAS
    return df

#Function to get the source page(html format) and parse it to beautifull soup
def scrap_data(indicator):
    url = 'http://iresearch.worldbank.org/PovcalNet/data.aspx' 
    driver = webdriver.Chrome() #calling selenium driver
    driver.get(url)
    select = Select(driver.find_element_by_class_name('topicTitle'))
    select.select_by_value(indicator) #selecting required attribute from dropdown using value (indicator)
    response = driver.page_source #getting html page using selenium driver
    soup = BeautifulSoup(response, "html.parser")
    driver.quit()
    return extract_data(soup) #passing argument as beautifull soup output to extarct data and required format

#Per Capita Expenditure Indicator function call
def Per_Capita_Expenditure():
    indicator = "PCE" 
    Per_Capita_Expenditure=scrap_data(indicator) 
    df_final_Per_Capita_Expenditure=pd.melt(Per_Capita_Expenditure, id_vars='Countries', value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
    df_final_Per_Capita_Expenditure.columns = ['Country','Year','Per_Capita_Expenditure']
    return df_final_Per_Capita_Expenditure

#GDP Indicator function call
def GDP():
    indicator = "GDP"
    GDP=scrap_data(indicator)
    df_final_GDP=pd.melt(GDP, id_vars='Countries', value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
    df_final_GDP.columns = ['Country','Year','GDP']
    return df_final_GDP

#Consumer Price Index Indicator function call
def Consumer_Price_Index():
    indicator = "CPI"
    Consumer_Price_Index=scrap_data(indicator) 
    df_final_Consumer_Price_Index=pd.melt(Consumer_Price_Index, id_vars='Countries', value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
    df_final_Consumer_Price_Index.columns = ['Country','Year','Consumer_Price_Index']
    return df_final_Consumer_Price_Index

#Currency Conversion function call
def Currency_Conversion():
    indicator = "CurrencyConversion"
    Currency_Conversion=scrap_data(indicator)
    df_final_Currency_Conversion=pd.melt(Currency_Conversion, id_vars='Countries', value_vars=['2010', '2011', '2012', '2013', '2014', '2015', '2016'])
    df_final_Currency_Conversion.columns = ['Country','Year','Currency_Conversion']
    return df_final_Currency_Conversion
    

Data is converted in Data Frames and comcatenated into one Data Frame

In [12]:
#concatenated different indicator's Data Frames
dfs = [Population(),Per_Capita_Expenditure(), GDP(), Consumer_Price_Index(),Currency_Conversion()]
df_final = reduce(lambda left,right: pd.merge(left,right,on=['Country', 'Year'], how='left'), dfs)
df_final

,Country,Year,Population,Per_Capita_Expenditure,GDP,Consumer_Price_Index,Currency_Conversion
0,American Samoa,2010,0.06,NaN,NaN,NaN,NaN
1,Cambodia,2010,14.31,NaN,NaN,0.9289,NaN
2,China,2010,"1,337.71","1,612.71","4,560.51",NaN,NaN
3,Fiji,2010,0.86,NaN,"3,651.97",n/a,NaN
4,Indonesia,2010,242.52,"1,750.31","3,113.48",0.9274,NaN
5,Kiribati,2010,0.10,NaN,"1,520.88",n/a,NaN
6,"Korea, Dem. People's Rep.",2010,24.59,NaN,NaN,NaN,NaN
7,Lao PDR,2010,6.25,851.19,"1,141.13",n/a,NaN
8,Malaysia,2010,28.11,"4,364.82","9,071.36",n/a,NaN
9,Marshall Islands,2010,0.05,NaN,NaN,NaN,NaN


Final Data Frame is saved in CSV file

In [11]:
#saved final data frame to CVS file.
df_final.to_csv("./data/Poverty-I.csv", index = False)